<a href="https://colab.research.google.com/github/kobrue02/evaluating-llm-generated-nlu-data/blob/main/bin/notebooks/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kobrue02/evaluating-llm-generated-nlu-data/
%cd evaluating-llm-generated-nlu-data/bin
!git pull

fatal: destination path 'evaluating-llm-generated-nlu-data' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
/content/evaluating-llm-generated-nlu-data/bin


In [2]:
from generate_data import DataGenerationModel
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
phi = DataGenerationModel(model=phi_model, tokenizer=phi_tokenizer)
prompt = [
                {"role": "system", "content": "You are an NLU expert, with a focus on NLU data generation."},
                {"role": "user", "content": "Can you generate 5 queries for the intent `ac_on`?"},
                {"role": "assistant", "content": "['Turn on the AC in the back of the car.', 'AC on', 'Put on the air con', 'Can you turn on AC?']"},
                {"role": "user", "content": f"How about 20 queries for the intent `turn_on_ac`? Please return the queries as a list of strings."},
            ]
phi_data = phi.generate_synthetic_data(prompt=prompt, num_samples=100)
phi_data

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


DataSet(data=[['Turn on the air conditioning in the living room.', 'AC on, please.', 'Activate the air conditioning in the bedroom.', 'Could you switch on the air con in the kitchen?', 'I need the air conditioning on in the study.', 'Please turn on the air conditioning in the bathroom.', 'Air conditioning, on please.', 'Activate the air conditioning in the garage.', 'Turn on the air conditioning in the office.', 'I need the air conditioning on in the dining room.', 'Please turn on the air conditioning in the hallway.', 'Activate the air conditioning in the laundry room.', 'Turn on the air conditioning in the basement.', 'Air conditioning, on please.', 'Activate the air conditioning in the attic.', 'Turn on the air conditioning in the guest room.', 'Please turn on the air conditioning in the guest bathroom.', 'Activate the air conditioning in the storage room.', 'Turn on the air conditioning in the pantry.', 'Air conditioning, on please.']], labels=['intent'])